**Generative Models for Code** -- Final Project<br><br>
**Maria Gancayco (mig2131@columbia.edu)**<br>
**Stephen Wright (svw2112@columbia.edu)**<br>
*Due:* Thursday, 19 Dec 2024 at 11:59pm ET

### Notebook Evaluation and Setup

The following notebook will provide all results when run sequentially for the following models: GPT-4, DeepSeek 7B, and Semcoder (10-30). A GPU must be used in order to generate new results. All needed results are printed to stdout with appropriate documentation.

The current setup is intended for Google Colab usage. To get results for GPT-4 in Google Colab, please provide an api key with the environment variable name OPENAI_API_KEY in Google Colab secrets. To reproduce novelty results using Claude as judge, please similarly add an ANTHROPIC_API_KEY in Google Colab secrets. For GCP VM environments, please use a shell environment variable with the same names (or if you're willing, you can just put the API key value in directly).

Please note that the numbers shown here may differ compared to what is in our report since we ran this notebook several times to ensure functional correctness post-cleanups before submission. SemCoder's reported dominant performance in the noted trends persisted through the multiple runs.




### Imports and Setup

In [1]:
# Setup: Environment and Memory Management

import torch
import gc
from pathlib import Path
from dataclasses import dataclass
from typing import Optional

# Check and display GPU availability for transparency
print("CUDA available:", torch.cuda.is_available())
print("GPU device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

# Memory management utilities
def clear_memory() -> None:
    """
    Clears GPU memory cache and performs garbage collection.

    This function is crucial for maintaining optimal memory usage during model evaluation,
    especially when loading and comparing multiple large language models.
    """
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # Clear CUDA cache
    gc.collect()  # Trigger Python garbage collection

def get_memory_status() -> None:
    """
    Displays current GPU memory usage statistics.

    Reports both allocated and reserved memory in megabytes (MB).
    This helps monitor memory consumption during model operations.

    Note:
        - Allocated memory: Actually used GPU memory
        - Reserved memory: Total memory reserved by PyTorch
    """
    if torch.cuda.is_available():
        # Convert bytes to MB for better readability
        allocated = torch.cuda.memory_allocated() / 1024**2
        reserved = torch.cuda.memory_reserved() / 1024**2
        print(f"GPU Memory: Allocated: {allocated:.2f}MB, Reserved: {reserved:.2f}MB")
clear_memory()
# Initialize by checking current memory status
get_memory_status()

CUDA available: True
GPU device name: NVIDIA A100-SXM4-40GB
GPU Memory: Allocated: 0.00MB, Reserved: 0.00MB


In [2]:
# Configuration and Setup

@dataclass
class ExperimentConfig:
    """
    Configuration dataclass containing all hyperparameters and settings for model evaluation.

    Attributes:
        model_name (str): Name/path of the model to be evaluated
        batch_size (int): Number of samples processed in each batch
        learning_rate (float): Learning rate for model optimization
        num_epochs (int): Number of training epochs
        max_seq_length (int): Maximum sequence length for input tokenization
        gradient_accumulation_steps (int): Number of steps to accumulate gradients
        warmup_steps (Optional[int]): Number of warmup steps for learning rate scheduler
        weight_decay (float): L2 regularization factor
        eval_steps (int): Frequency of evaluation steps
        save_steps (int): Frequency of model checkpoint saves
        logging_steps (int): Frequency of logging training metrics
    """
    model_name: str
    batch_size: int
    learning_rate: float
    num_epochs: int
    max_seq_length: int
    gradient_accumulation_steps: int
    warmup_steps: Optional[int] = None
    weight_decay: float = 0.01
    eval_steps: int = 100
    save_steps: int = 100
    logging_steps: int = 10

# Set up results directory for storing evaluation outputs
results_dir = Path("./results")
results_dir.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

print("Configuration and directories initialized!")

Configuration and directories initialized!


In [ ]:
config = ExperimentConfig(
    model_name="deepseek-ai/deepseek-coder-7b-instruct-v1.5",
    batch_size=1,                    # Small batch size due to model size
    learning_rate=5e-5,             # Conservative learning rate for fine-tuning
    num_epochs=3,                   # Number of training epochs
    max_seq_length=512,            # Maximum sequence length for input processing
    gradient_accumulation_steps=32, # Accumulate gradients to simulate larger batch size
    warmup_steps=100               # Warmup steps for learning rate scheduler
)

In [ ]:
deepseek_6_7b_config = ExperimentConfig(
    model_name="deepseek-ai/deepseek-coder-6.7b-instruct",
    batch_size=1,                    # Small batch size due to model size
    learning_rate=5e-5,             # Conservative learning rate for fine-tuning
    num_epochs=3,                   # Number of training epochs
    max_seq_length=512,            # Maximum sequence length for input processing
    gradient_accumulation_steps=32, # Accumulate gradients to simulate larger batch size
    warmup_steps=100               # Warmup steps for learning rate scheduler
)

In [31]:
# Model Dependencies and Imports

# Install core dependencies for transformer model handling and evaluation
!pip install transformers torch timeout-decorator
!pip install anthropic

# Import required libraries
import torch  # PyTorch for deep learning operations
from transformers import (
    AutoTokenizer,         # For tokenization of input text
    AutoModelForCausalLM   # For loading pre-trained causal language models
)
import timeout_decorator
!pip install datasets
from datasets import load_dataset
import numpy as np
from anthropic import Anthropic
import json
from google.colab import userdata
import os
from typing import Dict, List, Tuple

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from typing import Optional
import re

import tempfile
import subprocess
import statistics
import json
from pathlib import Path
from google.colab import files
!pip install openai==0.28
import openai
openai.api_key = userdata.get('OPENAI_API_KEY')
import pytest
import random
import concurrent.futures
from functools import partial
!pip install pytest pytest-cov coverage

In [4]:
dataset = load_dataset("openai_humaneval")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
# Model Loading and Code Generation

def load_model_and_tokenizer(config: ExperimentConfig) -> tuple[AutoModelForCausalLM, AutoTokenizer]:

    try:
        # Clear memory before loading new model to prevent OOM errors
        clear_memory()

        print(f"Loading {config.model_name}...")

        # Initialize tokenizer with remote code execution enabled
        tokenizer = AutoTokenizer.from_pretrained(
            config.model_name,
            trust_remote_code=True  # Required for custom tokenizer implementations
        )

        # Load model with memory-efficient settings
        model = AutoModelForCausalLM.from_pretrained(
            config.model_name,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,    # Use bfloat16 for memory efficiency
            device_map="auto",             # Optimize model placement across available devices
            low_cpu_mem_usage=True         # Minimize CPU memory during loading
        )

        # Enable gradient checkpointing if available
        if hasattr(model, "gradient_checkpointing_enable"):
            model.gradient_checkpointing_enable()  # Trade compute for memory savings

        print("Model loaded successfully!")
        get_memory_status()  # Display current memory usage

        return model, tokenizer

    except Exception as e:
        print(f"Error loading model: {str(e)}")
        raise

def generate_code(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    prompt: str,
    max_new_tokens: int = 512,
    temperature: float = 0.8,
    top_p: float = 0.95,
    top_k: int = 50
) -> str:

    try:
        # Format prompt as chat message
        messages = [{"role": "user", "content": prompt}]
        print("Generating inputs...")
        # Tokenize input with chat template
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)
        print("Generating outputs...")
        # Generate code with specified parameters
        outputs = model.generate(
            inputs,
            max_new_tokens=max_new_tokens,  # Control generation length
            do_sample=True,                 # Enable sampling-based generation
            temperature=temperature,         # Control randomness
            top_p=top_p,                    # Nucleus sampling threshold
            top_k=top_k,                    # Top-k sampling parameter
            num_return_sequences=1,         # Generate single sequence
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

        # Decode and return only the generated portion (excluding prompt)
        return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    except Exception as e:
        print(f"Error in code generation: {str(e)}")
        return ""


In [ ]:
# Initialize model and tokenizer using configuration
deepseek_7b_model, deepseek_7b_tokenizer = load_model_and_tokenizer(config)

Loading deepseek-ai/deepseek-coder-7b-instruct-v1.5...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully!
GPU Memory: Allocated: 13180.49MB, Reserved: 13182.00MB


In [ ]:
deepseek_6_7b_model, deepseek_6_7b_tokenizer = load_model_and_tokenizer(deepseek_6_7b_config)

Loading deepseek-ai/deepseek-coder-6.7b-instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!
GPU Memory: Allocated: 26037.02MB, Reserved: 26038.00MB


In [ ]:
"""
#################################
# SemCoder Model Setup
#################################
This section handles the installation and setup of the SemCoder model,
including Git LFS setup and repository cloning.
"""

# Clear GPU memory before new model setup
clear_memory()  # Ensure clean memory state for new model

# Install Git LFS and clone SemCoder repository
print("Installing Git LFS and cloning SemCoder...")
!git lfs install  # Initialize Git Large File Storage for model weights

# Clone SemCoder from HuggingFace repository
# Note: Using /content/SemCoder path for Google Colab compatibility
!git clone https://huggingface.co/semcoder/semcoder /content/SemCoder

if os.path.exists('/content/SemCoder'):
    print("SemCoder repository cloned successfully!")
else:
    raise RuntimeError("Failed to clone SemCoder repository")  # Critical error if clone fails

Installing Git LFS and cloning SemCoder...
Git LFS initialized.
fatal: destination path '/content/SemCoder' already exists and is not an empty directory.
SemCoder repository cloned successfully!


In [ ]:
"""
#################################
# SemCoder File Verification
#################################
This module verifies the integrity of the SemCoder installation by checking
for all required model files in the safetensors format.
"""

def verify_semcoder_files() -> None:
    """
    Verifies the presence of all required SemCoder model files.

    Checks for:
        - Configuration files (config.json, tokenizer.json)
        - Model weight files in safetensors format
        - Model index file

    Raises:
        RuntimeError: If any required files are missing from the installation
    """
    # Define required files for model functionality
    required_files = [
        'config.json',           # Model configuration
        'tokenizer.json',        # Tokenizer configuration
        'model.safetensors.index.json',  # Model weights index
        # Sharded model weights in safetensors format
        'model-00001-of-00003.safetensors',
        'model-00002-of-00003.safetensors',
        'model-00003-of-00003.safetensors'
    ]
    missing_files: List[str] = []

    # Display current directory contents for debugging
    print("SemCoder directory contents:")
    files = os.listdir('/content/SemCoder')
    print("\n".join(files))

    # Check for missing files
    for file in required_files:
        if file not in files:
            missing_files.append(file)

    # Handle verification results
    if missing_files:
        raise RuntimeError(f"Missing required files: {', '.join(missing_files)}")
    else:
        print("\nAll required files present!")
        print("\nModel files verification successful!")

# Execute verification
verify_semcoder_files()

SemCoder directory contents:
trainer_state.json
model.safetensors.index.json
tokenizer.json
model-00001-of-00003.safetensors
tokenizer_config.json
.git
generation_config.json
config.json
.gitattributes
special_tokens_map.json
training_args.bin
model-00003-of-00003.safetensors
model-00002-of-00003.safetensors
README.md

All required files present!

Model files verification successful!


In [ ]:
"""
#################################
# SemCoder Model Implementation
#################################
This module implements the SemCoder model class with memory-efficient loading
and code generation capabilities.
"""

class SemCoderModel:
    """
    A class implementing the SemCoder model with optimized loading and generation.

    Attributes:
        model_path (str): Path to the local SemCoder model files
        model: The loaded language model (initialized in load())
        tokenizer: The model's tokenizer (initialized in load())
    """

    def __init__(self, model_path: str):
        """
        Initialize SemCoder model instance.

        Args:
            model_path (str): Path to the local model directory
        """
        self.model_path = model_path
        self.model: Optional[AutoModelForCausalLM] = None
        self.tokenizer: Optional[AutoTokenizer] = None

    def load(self) -> None:
        """
        Load the SemCoder model and tokenizer with memory optimizations.

        Implements:
            - Memory clearing before load
            - bfloat16 precision for efficiency
            - Automatic device mapping
            - Gradient checkpointing

        Raises:
            Exception: If model loading fails
        """
        try:
            # Ensure clean memory state
            clear_memory()

            # Load tokenizer first
            print("Loading SemCoder tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

            # Load model with optimizations
            print("Loading SemCoder model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                torch_dtype=torch.bfloat16,    # Use bfloat16 for memory efficiency
                device_map="auto",             # Automatic device placement
                low_cpu_mem_usage=True         # Minimize CPU memory usage
            )

            # Enable memory optimization
            if hasattr(self.model, "gradient_checkpointing_enable"):
                self.model.gradient_checkpointing_enable()

            print("Successfully loaded SemCoder!")
            get_memory_status()  # Display memory usage

        except Exception as e:
            print(f"Error loading SemCoder: {str(e)}")
            raise

    def generate_code(self, prompt: str, max_new_tokens: int = 512) -> str:
        """
        Generate code using the loaded SemCoder model.

        Args:
            prompt (str): Input prompt for code generation
            max_new_tokens (int): Maximum number of tokens to generate

        Returns:
            str: Generated code or empty string if generation fails

        Note:
            Uses sampling-based generation with temperature=0.7 and top_p=0.95
            for balanced creativity and coherence
        """
        try:
            # Tokenize input with proper device placement
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                truncation=True
            ).to(self.model.device)

            # Generate with specified parameters
            outputs = self.model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=True,         # Enable sampling
                temperature=0.7,        # Control randomness
                top_p=0.95             # Nucleus sampling threshold
            )

            return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        except Exception as e:
            print(f"Error generating code: {str(e)}")
            return ""

# Initialize and load SemCoder model
semcoder = SemCoderModel("/content/SemCoder")
semcoder.load()

Loading SemCoder tokenizer...
Loading SemCoder model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Successfully loaded SemCoder!
GPU Memory: Allocated: 38255.52MB, Reserved: 38256.00MB


### General Utilities

In [57]:
def extract_test_suites(content: str) -> list[str]:
    """
    Extract test suites from the content and format them with function calls.
    Handles both standalone assert statements and function definitions.
    Returns a list of formatted test suite strings.
    """
    # Split content into test suite blocks
    test_blocks = re.split(r'Generated \d+ enhanced tests\nTotal tests so far: \d+/\d+\n+Generated tests:', content)

    # Remove empty blocks
    test_blocks = [block.strip() for block in test_blocks if block.strip()]

    formatted_suites = []
    for block in test_blocks:
        if "unittest.TestCase" in block:
          print("FORMATTED TEST SUITE:")
          print(block)
          formatted_suites.append(block)
          continue


        print("ORIGINAL TEST SUITE:")
        print(block)
        suite_parts = []

        # First, collect any imports at the start of the block
        import_statements = re.findall(r'^import [^\n]+', block, re.MULTILINE)

        # Extract function-based tests
        test_functions = re.finditer(r'def (test_\w+)\(\):\n((?:[ ]{4}.*\n?)+)', block)

        # Extract standalone assert statements (not within functions)
        # Looking for asserts that are at the start of a line and not indented
        standalone_asserts = re.finditer(r'^assert [^\n]+$', block, re.MULTILINE)

        # Extract standalone pytest.raises statements
        standalone_raises = re.finditer(r'^with pytest\.raises\([^\)]+\):\n[ ]{4}[^\n]+\n', block, re.MULTILINE)

        # Add imports if they exist
        if import_statements:
            suite_parts.extend(import_statements)
            suite_parts.append("")  # Add blank line after imports

        # Add standalone asserts
        for match in standalone_asserts:
            suite_parts.append(match.group(0))

        # Add standalone pytest.raises
        for match in standalone_raises:
            suite_parts.append(match.group(0).rstrip())

        # Add function-based tests
        for match in test_functions:
            func_name = match.group(1)
            func_body = match.group(2).rstrip()
            formatted_func = f"def {func_name}():\n{func_body}\n{func_name}()"
            suite_parts.append(formatted_func)

        if suite_parts:
            formatted_suite = "\n".join(suite_parts)
            print("FORMATTED TEST SUITE:")
            print(formatted_suite)
            print("-" * 50)
            formatted_suites.append(formatted_suite)

    return formatted_suites

def process_file_path(file_path: str) -> list[str]:
    """Process a file by path and return list of formatted test suite strings."""
    with open(file_path, 'r') as f:
        content = f.read()
    return extract_test_suites(content)

def process_file_content(content: str) -> list[str]:
    """Process file content directly and return list of formatted test suite strings."""
    return extract_test_suites(content)

def execute_test_case(code: str, test_case: str) -> bool:
    try:
        namespace = {}
        # Execute the function code
        exec(code, namespace)
        # Execute the test case
        exec("import pytest", namespace)
        exec(test_case, namespace)
        return True
    except pytest.raises.Exception:
        # This catches when pytest.raises() fails (i.e., expected exception wasn't raised)
        return False
    except Exception as e:
        # Catch any other exceptions
        return False

def check_syntax(code: str) -> bool:
    try:
        compile(code, '<string>', 'exec')
        return True
    except SyntaxError:
        return False

def _run_single_k_sample(solution: str, generated_tests: str, k: int) -> bool:
    """
    Run a single sample of k attempts and return True if any attempt succeeds.

    Args:
        solution: The code solution to test
        generated_tests: The test suite
        k: Number of attempts to consider

    Returns:
        bool: True if any attempt in k trials succeeds
    """
    for _ in range(k):
        if execute_test_case(solution, generated_tests):
            return True
    return False

def evaluate_pass_at_k(solution: str, generated_tests: str, k: int, num_samples: int = 1, max_workers: int = 4) -> float:
    """
    Evaluate pass@k metric by sampling k times and checking if any attempt succeeds.
    Uses parallel execution for multiple samples.

    Args:
        solution: The code solution to test
        generated_tests: The test suite
        k: Number of attempts to consider (e.g., 1 or 10)
        num_samples: Number of times to sample k attempts (default: 1)
        max_workers: Maximum number of parallel workers (default: 4)

    Returns:
        float: Pass@k success rate (0.0 to 1.0)
    """
    syntax_valid = check_syntax(solution + "\n" + generated_tests)
    if not syntax_valid:
        return 0.0

    # Create a partial function with solution and tests pre-filled
    run_sample = partial(_run_single_k_sample, solution, generated_tests, k)

    # Use ThreadPoolExecutor for parallel execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all samples to the executor
        future_results = [executor.submit(run_sample) for _ in range(num_samples)]

        # Collect results as they complete
        successes = sum(future.result() for future in concurrent.futures.as_completed(future_results))

    return successes / num_samples

@timeout_decorator.timeout(5)
def evaluate_single_test_suite(solution: str, generated_tests: str, num_samples: int = 1) -> Dict:
    """
    Evaluate a single test suite and return metrics including pass@1 and pass@10.
    Uses parallel execution for pass@k evaluation.

    Args:
        solution: The code solution to test
        generated_tests: The test suite
        num_samples: Number of samples for pass@k evaluation (default: 1)

    Returns:
        Dict containing evaluation metrics
    """
    syntax_valid = check_syntax(solution + "\n" + generated_tests)

    # Execute test cases if syntax is valid
    if syntax_valid:
        execution_success = execute_test_case(solution, generated_tests)
        # Evaluate pass@1 (single attempt) with parallel execution
        pass_1_result = float(evaluate_pass_at_k(solution, generated_tests, k=1, num_samples=num_samples))
        # Evaluate pass@10 (ten attempts) with parallel execution
        pass_10_result = float(evaluate_pass_at_k(solution, generated_tests, k=10, num_samples=num_samples))
    else:
        execution_success = False
        pass_1_result = 0.0
        pass_10_result = 0.0

    return {
        "syntax_valid": syntax_valid,
        "execution_success": execution_success,
        "pass@1": pass_1_result,
        "pass@10": pass_10_result
    }

def evaluate_test_suite(model_type, dataset, n_tasks, test_suites):
    solutions = dataset['test']["canonical_solution"]
    metrics = {"syntax_validity": 0.0,  # Syntactic correctness
              "execution_accuracy": 0.0,  # Functional correctness
              "pass@1": 0.0,
              "pass@10": 0.0
    }
    results = []
    with open(f'{model_type}_test_case_generation_accuracy_results.txt', 'w') as f:
            for i in range(n_tasks):
                solution = solutions[i]
                full_solution = dataset['test']["prompt"][i] + solution
                cleaned_tests = test_suites[i]
                try:
                  result = evaluate_single_test_suite(full_solution, cleaned_tests)
                except Exception as e:
                  print(f"Error evaluating test suite: {str(e)}")
                  result = {
                      "syntax_valid": False,
                      "execution_success": False,
                      "pass@1": 0.0,
                      "pass@10": 0.0
                  }


                f.write(f"PROBLEM {i}:\n")
                print(f"PROBLEM {i}:\n")
                f.write("CANONICAL SOLUTION:\n")
                print("CANONICAL SOLUTION:\n")
                f.write(full_solution + "\n")
                print(full_solution + "\n")
                f.write("CLEANED TESTS:\n")
                print("CLEANED TESTS:\n")
                f.write(cleaned_tests + "\n")
                print(cleaned_tests)
                f.write("RESULT:\n" + str(result) + "\n")
                print("RESULT:\n" + str(result))

                results.append(result)

            metrics["syntax_validity"] = np.mean([r["syntax_valid"] for r in results])
            metrics["execution_accuracy"] = np.mean([r["execution_success"] for r in results])
            metrics["pass@1"] = np.mean([r["pass@1"] for r in results])
            metrics["pass@10"] = np.mean([r["pass@10"] for r in results])
            print(metrics)
            f.write(str(metrics))

def clean_deepseek_generated_code(code: str) -> str:
        """Clean up generated code to extract only the functions."""
        lines = code.split('\n')
        cleaned_lines = []
        found_start = False
        found_test_func_call = False
        for line in lines:
            if line.startswith('```python'):
                found_start = True
            elif line.startswith('```'):
                if found_test_func_call: break
                else: found_start = False
            elif found_start:
                if line.startswith('test_') and line.endswith('()'):
                    found_test_func_call = True
                cleaned_lines.append(line)

        return '\n'.join(cleaned_lines).strip()

def clean_semcoder_generated_code(code: str) -> str:
    """Clean up generated code to extract only the functions."""
    lines = code.split('\n')
    cleaned_lines = []
    in_function = False

    for line in lines:
        if line.strip().startswith('def '):
            in_function = True
            cleaned_lines.append(line)
        elif in_function and (line.startswith('    ') or not line.strip()):
            cleaned_lines.append(line)
        elif in_function and line.strip() and not line.startswith('    '):
            in_function = False
            cleaned_lines.append('')

    return '\n'.join(cleaned_lines).strip()

### HumanEval Test Case Generation

In [29]:
def generate_humaneval_tests(model_type, deepseek_model=None, deepseek_tokenizer=None, num_total_tests=100):
    dataset = load_dataset("openai_humaneval")
    results = []
    total_tests_generated = 0
    with open(f'{model_type}_test_case_generation_results.txt', 'w') as f:
      for i in range(len(dataset['test'])):
          if total_tests_generated >= num_total_tests:
              break

          problem = dataset['test'][i]
          prompt = problem['prompt']
          solution = problem['canonical_solution']
          entry_point = problem['entry_point']
          test_code = problem['test']

          # Extract working test cases
          check_match = re.search(r'def check\(candidate\):\s*(.*?)(?=\n\n|$)', test_code, re.DOTALL)
          test_cases = re.findall(r'assert.*?(?=\n|$)', check_match.group(1) if check_match else '')

          test_prompt = f"""
Please provide executable test cases for this function:
{prompt}

Working test examples:
{test_cases}

Include these types of tests:
1. Performance test:
def test_{entry_point}_perf():
    {test_cases[0].replace('candidate', entry_point)}

2. Edge case test:
def test_{entry_point}_edge():
    {test_cases[-1].replace('candidate', entry_point)}

3. Error test:
def test_{entry_point}_error():
    with pytest.raises(TypeError):
        {entry_point}(None)

Only provide executable test cases. No placeholders."""

          try:
              generated_tests, cleaned_tests = None, None
              if model_type == "semcoder":
                generated_tests = semcoder.generate_code(test_prompt)
                cleaned_tests = clean_semcoder_generated_code(generated_tests)
              elif "deepseek"in model_type:
                generated_tests = generate_code(deepseek_model, deepseek_tokenizer, test_prompt, max_new_tokens=4096)
                cleaned_tests = clean_deepseek_generated_code(generated_tests)
              elif model_type == "gpt-4":
                response = openai.ChatCompletion.create(
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": test_prompt}
                    ],
                    max_tokens=4096,
                    temperature=0.8,
                    top_p=0.95
                )
                generated_tests = response["choices"][0]["message"]["content"].strip()
                cleaned_tests = clean_deepseek_generated_code(generated_tests) #TODO:- Please rename this since we're also using for OpenAI
              if cleaned_tests:
                  num_tests = len(re.findall(r'def test_', cleaned_tests))
                  total_tests_generated += num_tests

                  result = {
                      'problem_id': i,
                      'entry_point': entry_point,
                      'tests': cleaned_tests,
                      'num_tests': num_tests
                  }
                  results.append(result)

                  print(f"Generated {num_tests} enhanced tests")
                  print(f"Total tests so far: {total_tests_generated}/{num_total_tests}")
                  print("\nTest prompt:")
                  print(test_prompt)
                  print("\nGenerated tests:")
                  print(generated_tests)
                  print("\nCleaned tests:")
                  print(cleaned_tests)

                  f.write(f"Generated {num_tests} enhanced tests\n")
                  f.write(f"Total tests so far: {total_tests_generated}/{num_total_tests}")
                  f.write("\nGenerated tests:\n")
                  f.write(cleaned_tests + "\n")
              else:
                  print("No valid tests generated")

          except Exception as e:
              print(f"Error generating tests: {str(e)}")
              continue

    return results, total_tests_generated

In [ ]:
# Generate HumanEval tests for DeepSeek 7B
print("Generating HumanEval test cases...")
plus_results, total_plus_tests = generate_humaneval_tests("deepseek_7b", deepseek_model=deepseek_7b_model, deepseek_tokenizer=deepseek_7b_tokenizer, num_total_tests=100)

Generating HumanEval test cases...
Generating inputs...
Generating outputs...
Generated 4 enhanced tests
Total tests so far: 4/100

Test prompt:

Please provide executable test cases for this function:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


Working test examples:
['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True', 'assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False', 'assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 

In [ ]:
# Generate HumanEval tests for DeepSeek 6.7B
print("Generating HumanEval test cases...")
plus_results, total_plus_tests = generate_humaneval_tests("deepseek_6_7b", deepseek_model=deepseek_6_7b_model, deepseek_tokenizer=deepseek_6_7b_tokenizer, num_total_tests=100)

Generating HumanEval test cases...
Generating inputs...
Generating outputs...
Generated 0 enhanced tests
Total tests so far: 0/100

Test prompt:

Please provide executable test cases for this function:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


Working test examples:
['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True', 'assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False', 'assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 

In [ ]:
# Generate HumanEval tests for SemCoder
print("Generating HumanEval test cases...")
plus_results, total_plus_tests = generate_humaneval_tests("semcoder", num_total_tests=100)

Generating HumanEval test cases...


Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Generated 3 enhanced tests
Total tests so far: 3/100

Test prompt:

Please provide executable test cases for this function:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


Working test examples:
['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True', 'assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False', 'assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False']

Include these types of tests:
1. Performance test:
def test_h

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Generated 3 enhanced tests
Total tests so far: 6/100

Test prompt:

Please provide executable test cases for this function:
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """


Working test examples:
["assert candidate('(()()) ((())) () ((())()())') == [", "assert candidate('() (()) ((())) (((())))') == [", "assert candidate('(()(())((())))') == [", "assert candidate('( ) (( )) (( )( ))') == ['()', '(())', '(()())']"]

Include these types of tests:
1. Performance test:
def test_separate_paren_groups_perf():
    assert separate_paren_groups

...

Generated 3 enhanced tests
Total tests so far: 93/100

Test prompt:

Please provide executable test cases for this function:


def get_positive(l: list):
    """Return only positive numbers in the list.
    >>> get_positive([-1, 2, -4, 5, 6])
    [2, 5, 6]
    >>> get_positive([5, 3, -5, 2, -3, 3, 9, 0, 123, 1, -10])
    [5, 3, 2, 3, 9, 123, 1]
    """


Working test examples:
['assert candidate([-1, -2, 4, 5, 6]) == [4, 5, 6]', 'assert candidate([5, 3, -5, 2, 3, 3, 9, 0, 123, 1, -10]) == [5, 3, 2, 3, 3, 9, 123, 1]', 'assert candidate([-1, -2]) == []', 'assert candidate([]) == []']

Include these types of tests:
1. Performance test:
def test_get_positive_perf():
    assert get_positive([-1, -2, 4, 5, 6]) == [4, 5, 6]

2. Edge case test:
def test_get_positive_edge():
    assert get_positive([]) == []

3. Error test:
def test_get_positive_error():
    with pytest.raises(TypeError):
        get_positive(None)

Only provide executable test cases. No placeholders.

Generated tests:

Please 

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Generated 3 enhanced tests
Total tests so far: 96/100

Test prompt:

Please provide executable test cases for this function:


def is_prime(n):
    """Return true if a given number is prime, and false otherwise.
    >>> is_prime(6)
    False
    >>> is_prime(101)
    True
    >>> is_prime(11)
    True
    >>> is_prime(13441)
    True
    >>> is_prime(61)
    True
    >>> is_prime(4)
    False
    >>> is_prime(1)
    False
    """


Working test examples:
['assert candidate(6) == False', 'assert candidate(101) == True', 'assert candidate(11) == True', 'assert candidate(13441) == True', 'assert candidate(61) == True', 'assert candidate(4) == False', 'assert candidate(1) == False', 'assert candidate(5) == True', 'assert candidate(11) == True', 'assert candidate(17) == True', 'assert candidate(5 * 17) == False', 'assert candidate(11 * 7) == False', 'assert candidate(13441 * 19) == False']

Include these types of tests:
1. Performance test:
def test_is_prime_perf():
    assert is_prime(6) =

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Generated 3 enhanced tests
Total tests so far: 99/100

Test prompt:

Please provide executable test cases for this function:
import math


def poly(xs: list, x: float):
    """
    Evaluates polynomial with coefficients xs at point x.
    return xs[0] + xs[1] * x + xs[1] * x^2 + .... xs[n] * x^n
    """
    return sum([coeff * math.pow(x, i) for i, coeff in enumerate(xs)])


def find_zero(xs: list):
    """ xs are coefficients of a polynomial.
    find_zero find x such that poly(x) = 0.
    find_zero returns only only zero point, even if there are many.
    Moreover, find_zero only takes list xs having even number of coefficients
    and largest non zero coefficient as it guarantees
    a solution.
    >>> round(find_zero([1, 2]), 2) # f(x) = 1 + 2x
    -0.5
    >>> round(find_zero([-6, 11, -6, 1]), 2) # (x - 1) * (x - 2) * (x - 3) = -6 + 11x - 6x^2 + x^3
    1.0
    """


Working test examples:
['assert math.fabs(poly(coeffs, solution)) < 1e-4']

Include these types of tests:
1. Perfo

### Accuracy Results: DeepSeek 6.7B, 7B, vs SemCoder

In [58]:
deepseek_7b_extracted_test_suites = process_file_path("/content/deepseek_7b_test_case_generation_results.txt")

ORIGINAL TEST SUITE:
import pytest
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    """
    pass

def test_has_close_elements():
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False

def test_has_close_elements_perf():
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True

def test_has_close_elements_edge():
    assert has_close_elements([1.1, 2.2, 3.1, 4.1,

In [59]:
deepseek_6_7b_extracted_test_suites = process_file_path("/content/deepseek_6_7b_test_case_generation_results.txt")

ORIGINAL TEST SUITE:
# Performance test
assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True

# Edge case test
assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == False

# Error test
with pytest.raises(TypeError):
    has_close_elements(None)
FORMATTED TEST SUITE:
assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == False
--------------------------------------------------
ORIGINAL TEST SUITE:
def test_separate_paren_groups_perf():
    assert separate_paren_groups('(()()) ((())) () ((())()())') == ['()', '(())', '(()())', '((()))', '(((()())())))']

def test_separate_paren_groups_edge():
    assert separate_paren_groups('( ) (( )) (( )( ))') == ['()', '(())', '(()())']

def test_separate_paren_groups_error():
    with pytest.raises(TypeError):
        separate_paren_groups(None)
FORMATTED TEST SUITE:
def test_separate_paren_groups_perf():
    assert separate_paren_groups('(()()) (((

In [60]:
semcoder_extracted_test_suites = process_file_path("/content/semcoder_test_case_generation_results.txt")

ORIGINAL TEST SUITE:
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



def test_has_close_elements_perf():
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True


def test_has_close_elements_edge():
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False


def test_has_close_elements_error():
    with pytest.raises(TypeError):
        has_close_elements(None)
FORMATTED TEST SUITE:
def test_has_close_elements_perf():
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
test_has_close_elements_perf()
def test_has_close_elements_edge():
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False
test_has_close_elements_edge()
def test_has_close_elemen

In [61]:
evaluate_test_suite("deepseek_7b", dataset, len(deepseek_7b_extracted_test_suites), deepseek_7b_extracted_test_suites)

PROBLEM 0:

CANONICAL SOLUTION:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


CLEANED TESTS:

import pytest

def test_has_close_elements():
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    asser

In [62]:
evaluate_test_suite("deepseek_6_7b", dataset, len(deepseek_6_7b_extracted_test_suites), deepseek_6_7b_extracted_test_suites)

PROBLEM 0:

CANONICAL SOLUTION:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


CLEANED TESTS:

assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == False
RESULT:
{'syntax_valid': True, 'execution_success': False, 'pass@1': 0.0, 'pass@10': 0.0}
PROBLEM 1:

CANONICAL SOLUTION:

from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
   

In [63]:
evaluate_test_suite("semcoder", dataset, len(semcoder_extracted_test_suites), semcoder_extracted_test_suites)

PROBLEM 0:

CANONICAL SOLUTION:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


CLEANED TESTS:

def test_has_close_elements_perf():
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
test_has_close_elements_perf()
def test_has_close_elements_edge():
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False
test_has_close_elements_edge()
def test_has_close_elements_error():
    with pytest.raises(TypeError

### DeepSeek vs. SemCoder Prompt Sensitivity Effects

This section is dedicated to an ablative study to analyze the effects of using a simple prompt vs. a template-based prompt. We find based on these outputs, SemCoder requires a template-based prompt with real problem values injected. Here we specifically see that if we use a simple prompt and not a template-based prompt that uses real values, SemCoder gets confused/does not have enough guidance for the problem at hand and begins repeating prompt information and not responding to our directions. The prompt regurgitation may be an effect of SemCoder's forward monologue fine-tuning/training.

Based on these results, we use template-based prompts for our main experiments.

In [ ]:
metrics = {"syntax_validity": 0.0,  # Syntactic correctness
            "execution_accuracy": 0.0  # Functional correctness
}
def evaluate_model(model, dataset, model_type, tokenizer=None, n_tasks: int = None):
        solutions = dataset['test']["canonical_solution"]
        if n_tasks is None:
            n_tasks = len(solutions)

        results = []
        with open(f'{model_type}_test_case_generation_results_with_simple_prompt.txt', 'w') as f:
          for i in range(n_tasks):
              solution = solutions[i]
              full_solution = dataset['test']["prompt"][i] + solution

              prompt = f"""
              Please provide and execute a set of test cases for the following function:
              {full_solution}

              Please do not include natural language or anything that cannot be compiled/executed.
              Please only provided the test cases and their immediate execution.

              Example:
              def test_hello_with_name():
                  assert hello("Alice") == "Hello, Alice"
                  assert hello("Bob") == "Hello, Bob"
              test_hello_with_name()

              def test_hello_without_name():
                  assert hello(None) == "Hello, world"
                  assert hello("") == "Hello, world"
              test_hello_without_name()
              """
              generated_tests = ""
              if model_type == "deepseek":
                  generated_tests = generate_code(
                      model,
                      tokenizer,
                      prompt,
                      max_new_tokens=4096
                  )
              elif model_type == "semcoder":
                  generated_tests = model.generate_code(prompt, max_new_tokens=4096)

              cleaned_tests = clean_deepseek_generated_code(generated_tests) if model_type == "deepseek" else "" #no-op for now
              result = evaluate_single_test_suite(full_solution, cleaned_tests)

              f.write(f"PROBLEM {i}:\n")
              print(f"PROBLEM {i}:\n")
              f.write("CANONICAL SOLUTION:\n")
              print("CANONICAL SOLUTION:\n")
              f.write(full_solution + "\n")
              print(full_solution + "\n")
              f.write("GENERATED TESTS:\n")
              print("GENERATED TESTS:\n")
              f.write(generated_tests + "\n")
              print(generated_tests)
              f.write("CLEANED TESTS:\n")
              print("CLEANED TESTS:\n")
              f.write(cleaned_tests + "\n")
              print(cleaned_tests)
              f.write("RESULT:\n" + str(result) + "\n")
              print("RESULT:\n" + str(result))

              results.append(result)

          # Calculate aggregate metrics
          metrics["syntax_validity"] = np.mean([r["syntax_valid"] for r in results])
          metrics["execution_accuracy"] = np.mean([r["execution_success"] for r in results])
          f.write(str(metrics))
        return metrics

In [ ]:
metrics = evaluate_model(semcoder, dataset, "semcoder", tokenizer=None, n_tasks=100)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


PROBLEM 0:

CANONICAL SOLUTION:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


GENERATED TESTS:


              Please provide and execute a set of test cases for the following function:
              from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


PROBLEM 1:

CANONICAL SOLUTION:

from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """
    result = []
    current_string = []
    current_depth = 0

    for c in paren_string:
        if c == '(':
            current_depth += 1
            current_string.append(c)
        elif c == ')':
            current_depth -= 1
            current_string.append(c)

            if current_depth == 0:
                result.append(''.join(current_string))
                current_string.clear()

    return result


GENERATED TESTS:


              Please 

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


...

PROBLEM 96:

CANONICAL SOLUTION:


def count_up_to(n):
    """Implement a function that takes an non-negative integer and returns an array of the first n
    integers that are prime numbers and less than n.
    for example:
    count_up_to(5) => [2,3]
    count_up_to(11) => [2,3,5,7]
    count_up_to(0) => []
    count_up_to(20) => [2,3,5,7,11,13,17,19]
    count_up_to(1) => []
    count_up_to(18) => [2,3,5,7,11,13,17]
    """
    primes = []
    for i in range(2, n):
        is_prime = True
        for j in range(2, i):
            if i % j == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(i)
    return primes



GENERATED TESTS:


              Please provide and execute a set of test cases for the following function:
              
def count_up_to(n):
    """Implement a function that takes an non-negative integer and returns an array of the first n
    integers that are prime numbers and less than n.
    for example:
    count

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


PROBLEM 97:

CANONICAL SOLUTION:


def multiply(a, b):
    """Complete the function that takes two integers and returns 
    the product of their unit digits.
    Assume the input is always valid.
    Examples:
    multiply(148, 412) should return 16.
    multiply(19, 28) should return 72.
    multiply(2020, 1851) should return 0.
    multiply(14,-15) should return 20.
    """
    return abs(a % 10) * abs(b % 10)


GENERATED TESTS:


              Please provide and execute a set of test cases for the following function:
              
def multiply(a, b):
    """Complete the function that takes two integers and returns 
    the product of their unit digits.
    Assume the input is always valid.
    Examples:
    multiply(148, 412) should return 16.
    multiply(19, 28) should return 72.
    multiply(2020, 1851) should return 0.
    multiply(14,-15) should return 20.
    """
    return abs(a % 10) * abs(b % 10)


              Please do not include natural language or anything that cann

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


PROBLEM 98:

CANONICAL SOLUTION:


def count_upper(s):
    """
    Given a string s, count the number of uppercase vowels in even indices.
    
    For example:
    count_upper('aBCdEf') returns 1
    count_upper('abcdefg') returns 0
    count_upper('dBBE') returns 0
    """
    count = 0
    for i in range(0,len(s),2):
        if s[i] in "AEIOU":
            count += 1
    return count


GENERATED TESTS:


              Please provide and execute a set of test cases for the following function:
              
def count_upper(s):
    """
    Given a string s, count the number of uppercase vowels in even indices.
    
    For example:
    count_upper('aBCdEf') returns 1
    count_upper('abcdefg') returns 0
    count_upper('dBBE') returns 0
    """
    count = 0
    for i in range(0,len(s),2):
        if s[i] in "AEIOU":
            count += 1
    return count


              Please do not include natural language or anything that cannot be compiled/executed.
              Please only pro

 ### Code Line Coverage Assessment

In [15]:
def calculate_aggregate_metrics(results, target_score_name) -> Dict:
    if not results:
        return {'error': 'No valid results to analyze'}

    score_values = [r[target_score_name] for r in results if target_score_name in r]

    if not score_values:
        return {'error': 'No valid score values found'}

    return {
        f'mean_{target_score_name}': statistics.mean(score_values),
        f'median_{target_score_name}': statistics.median(score_values),
        f'min_{target_score_name}': min(score_values),
        f'max_{target_score_name}': max(score_values),
        f'std_dev': statistics.stdev(score_values) if len(score_values) > 1 else 0,
        'total_entries_analyzed': len(score_values)
    }

In [16]:
class TestCoverageAnalyzer:
    def __init__(self, input_file: str = "", output_dir: str = "/content/coverage_results"):
        """Initialize the analyzer with input file path and output directory."""
        self.input_file = input_file
        self.output_dir = output_dir
        self.coverage_results = []
        os.makedirs(output_dir, exist_ok=True)

    def create_test_files(self, solution: str, tests: str, temp_dir: str) -> Tuple[str, str]:
        """Create temporary Python files for the solution and tests."""
        # Create solution file
        solution_file = Path(temp_dir) / "solution.py"
        with open(solution_file, 'w') as f:
            f.write(solution)

        # Create test file with proper imports for Colab
        test_file = Path(temp_dir) / "test_solution.py"
        with open(test_file, 'w') as f:
            f.write("import sys\n")
            f.write(f"sys.path.append('{temp_dir}')\n")
            f.write("from solution import *\n")
            f.write(tests)

        return str(solution_file), str(test_file)

    def get_coverage_data(self):
        with open('coverage.json') as f:
            coverage_data = json.load(f)
            for file_path, file_data in coverage_data['files'].items():
                  if 'solution.py' in file_path:
                     return {
                        'line_coverage': file_data['summary']['percent_covered'],
                        'total_lines': file_data['summary']['num_statements'],
                        'covered_lines': file_data['summary']['covered_lines'],
                        'missing_lines': file_data['summary']['missing_lines']
                     }
    def run_coverage_analysis(self, solution_file: str, test_file: str, temp_dir: str) -> Dict:
        """Run pytest with coverage and return results."""
        try:
            orig_dir = os.getcwd()
            os.chdir(temp_dir)

            # Run pytest with coverage using python -m to ensure proper module resolution
            cmd = ['python3', '-m', 'pytest', '--cov=solution',
                '--cov-report=json', 'test_solution.py', '-v']

            env = os.environ.copy()
            env['PYTHONPATH'] = temp_dir  # Ensure proper module resolution

            result = subprocess.run(cmd, capture_output=True, text=True, env=env)
            if os.path.exists('coverage.json'): return self.get_coverage_data()
            return {'error': 'No coverage data generated'}

        except subprocess.CalledProcessError as e:
            print(f"Command output: {e.output}")
            return {'error': f'pytest failed: {str(e)}'}
        except Exception as e:
            print(f"Exception details: {str(e)}")
            return {'error': f'Analysis failed: {str(e)}'}
        finally:
            os.chdir(orig_dir)

In [41]:
coverage_analyzer = TestCoverageAnalyzer()

In [40]:
def print_coverage_results(coverage_analyzer, dataset, test_suites):
    coverage_results = []
    for index, test_suite in enumerate(test_suites):
      solution = dataset['test']["prompt"][index] + dataset['test']["canonical_solution"][index]
      with tempfile.TemporaryDirectory() as temp_dir:
        solution_file, test_file = coverage_analyzer.create_test_files(solution, test_suite, temp_dir)
        result = coverage_analyzer.run_coverage_analysis(solution_file, test_file, temp_dir)
        if 'line_coverage' in result:
          coverage_results.append(result)
    print(calculate_aggregate_metrics(coverage_results, "line_coverage"))

#### Line Coverage Results: DeepSeek vs. SemCoder

In [64]:
coverage_analyzer = TestCoverageAnalyzer()
print_coverage_results(coverage_analyzer, dataset, deepseek_7b_extracted_test_suites)

{'mean_line_coverage': 98.61111111111111, 'median_line_coverage': 100.0, 'min_line_coverage': 66.66666666666667, 'max_line_coverage': 100.0, 'std_dev': 6.804138174397716, 'total_entries_analyzed': 24}


In [ ]:
print_coverage_results(coverage_analyzer, dataset, deepseek_6_7b_extracted_test_suites)

{'mean_line_coverage': 65.15114515114516, 'median_line_coverage': 66.66666666666667, 'min_line_coverage': 13.333333333333334, 'max_line_coverage': 100.0, 'std_dev': 32.8185868519849, 'total_entries_analyzed': 33}


In [ ]:
print_coverage_results(coverage_analyzer, dataset, semcoder_extracted_test_suites)

{'mean_line_coverage': 96.75324675324676, 'median_line_coverage': 100.0, 'min_line_coverage': 21.428571428571427, 'max_line_coverage': 100.0, 'std_dev': 14.40695307294402, 'total_entries_analyzed': 33}


### Measuring Novelty and Diversity

#### Measuring with LLM as Judge

In [12]:
def analyze_novelty_with_claude(source_function: str, generated_tests: str, original_tests: str = None) -> dict:
    anthropic = Anthropic(api_key=userdata.get('ANTHROPIC_API_KEY'))

    prompt = f"""
As an expert test engineer, analyze the semantic novelty and diversity of the generated test cases for the given function. Consider the function's purpose, edge cases, and expected behaviors.

Source Function:

{source_function}


Generated Test Suite:

{generated_tests}

Original Test Suite:

{original_tests}

Please analyze:
1. How well do the tests cover different aspects of the function's behavior?
2. What novel testing scenarios are introduced?
3. Are there important edge cases or boundary conditions tested?
4. How diverse are the test inputs and scenarios?
5. Are the tests relevant to the function's purpose?

Provide your analysis in the following JSON format:
{{
    "novelty_score": <float between 0.0 and 1.0>,
    "novel_aspects": [<list of strings describing novel aspects>],
    "unique_scenarios": [<list of strings describing unique test scenarios>],
    "coverage_assessment": <string describing overall test coverage>,
    "recommendations": [<list of strings with suggested additional test cases>]
}}
Do not provide any other additonal text other than the JSON in order to facilitate
text processing.

"""

    message = anthropic.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        temperature=0,  # Use 0 for consistent analysis
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )

    try:
        # Parse the response as JSON
        analysis = json.loads(message.content[0].text)
        return analysis
    except json.JSONDecodeError:
        print("Failed to parse Claude's response as JSON")
        return None

In [13]:
def get_novelty_results(dataset, test_suites):
  novelty_results = []
  for index, test_suite in enumerate(test_suites):
    solution = dataset['test']["prompt"][index] + dataset['test']["canonical_solution"][index]
    original_tests = dataset['test']["test"][index]
    result = analyze_novelty_with_claude(solution, test_suite, original_tests)
    novelty_results.append(result)
  return novelty_results

Novelty Results with Claude as Judge: DeepSeek vs. SemCoder

In [17]:
deepseek_7b_novelty_results = get_novelty_results(dataset, deepseek_7b_extracted_test_suites)
print(calculate_aggregate_metrics(deepseek_7b_novelty_results, "novelty_score"))

{'mean_novelty_score': 0.625, 'median_novelty_score': 0.6499999999999999, 'min_novelty_score': 0.4, 'max_novelty_score': 0.7, 'std_dev': 0.09890707100936803, 'total_entries_analyzed': 24}


In [18]:
deepseek_6_7b_novelty_results = get_novelty_results(dataset, deepseek_6_7b_extracted_test_suites)
print(calculate_aggregate_metrics(deepseek_6_7b_novelty_results, "novelty_score"))

{'mean_novelty_score': 0.4303030303030303, 'median_novelty_score': 0.3, 'min_novelty_score': 0.2, 'max_novelty_score': 0.7, 'std_dev': 0.21863904112264645, 'total_entries_analyzed': 33}


In [19]:
semcoder_novelty_results = get_novelty_results(dataset, semcoder_extracted_test_suites)
print(calculate_aggregate_metrics(semcoder_novelty_results, "novelty_score"))

{'mean_novelty_score': 0.8, 'median_novelty_score': 0.8, 'min_novelty_score': 0.8, 'max_novelty_score': 0.8, 'std_dev': 0, 'total_entries_analyzed': 1}


In [20]:
import json

# Assuming deep_seek_novelty_results and semcoder_novelty_results are lists of dictionaries
# as produced by your analyze_novelty_with_claude function.


def write_results_to_file(results, filename):
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)


write_results_to_file(deepseek_7b_novelty_results, 'deep_seek_7b_novelty_results.json')
write_results_to_file(semcoder_novelty_results, 'semcoder_novelty_results.json')
write_results_to_file(deepseek_6_7b_novelty_results, 'deep_seek_6_7b_novelty_results.json')

from google.colab import files

files.download('deep_seek_7b_novelty_results.json')
files.download('semcoder_novelty_results.json')
files.download('deep_seek_6_7b_novelty_results.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Measuring Novelty with Patterns

In [21]:
from typing import Dict, List
import re
from collections import defaultdict

class CoveragePatternAnalyzer:
    """Analyzes test coverage patterns focusing on types of test cases."""

    def __init__(self):
        self.patterns = {
            'edge_cases': {
                'empty_input': r'assert.*(?:empty|\[\]|\{\}|\(\)|""|\'\'|\b==\s*\[\]|\b==\s*""|\b==\s*\'\')',
                'null_input': r'assert.*(?:None|null)',
                'single_element': r'assert.*\[[^,\]]+\]'
            },
            'boundary_testing': {
                'zero_values': r'assert.*\b0\b',
                'negative_values': r'assert.*-\d+',
                'large_values': r'assert.*\d{5,}'
            },
            'error_handling': {
                'exception_testing': r'with\s+pytest\.raises\([^)]+\)',
                'invalid_input': r'assert.*(invalid|wrong|incorrect|bad)'
            },
            'functionality': {
                'typical_case': r'assert.*normal|typical|standard',
                'complex_input': r'assert.*(?:\[.*,.*,.*\]|\{.*:.*,.*:.*\}|\(.*,.*,.*\))'
            }
        }

    def _extract_assertions(self, test_code: str) -> List[str]:
        """Extract assertions with improved handling of multi-line and truncated assertions."""
        lines = test_code.split('\n')
        assertions = []
        current_assertion = None
        in_raises_block = False
        bracket_count = 0
        paren_count = 0

        for line in lines:
            line = line.strip()

            # Skip empty lines
            if not line:
                continue

            # Start of pytest.raises block
            if 'pytest.raises' in line:
                in_raises_block = True
                current_assertion = line
                paren_count = line.count('(') - line.count(')')
                if paren_count == 0:
                    assertions.append(current_assertion)
                    current_assertion = None
                    in_raises_block = False
                continue

            # Start of regular assertion
            if line.startswith('assert'):
                current_assertion = line
                bracket_count = line.count('[') - line.count(']')
                paren_count = line.count('(') - line.count(')')
                if bracket_count == 0 and paren_count == 0:
                    assertions.append(current_assertion)
                    current_assertion = None
                continue

            # Continue previous assertion
            if current_assertion:
                current_assertion += ' ' + line
                if in_raises_block:
                    paren_count += line.count('(') - line.count(')')
                    if paren_count == 0:
                        assertions.append(current_assertion)
                        current_assertion = None
                        in_raises_block = False
                else:
                    bracket_count += line.count('[') - line.count(']')
                    paren_count += line.count('(') - line.count(')')
                    if bracket_count == 0 and paren_count == 0:
                        assertions.append(current_assertion)
                        current_assertion = None

        # Handle any remaining incomplete assertion
        if current_assertion:
            assertions.append(current_assertion + ' ...')

        return assertions

    def analyze_test_suite(self, test_code: str) -> Dict:
        """Analyze a test suite and return detailed coverage metrics."""
        assertions = self._extract_assertions(test_code)
        total_assertions = len(assertions)
        if total_assertions == 0:
            return {'error': 'No assertions found'}

        # Track which patterns match each assertion
        assertion_patterns = {i: set() for i in range(total_assertions)}
        pattern_counts = defaultdict(lambda: defaultdict(int))
        uncategorized_assertions = []

        # Analyze each assertion
        for i, assertion in enumerate(assertions):
            matches_found = False
            for category, patterns in self.patterns.items():
                for name, pattern in patterns.items():
                    if re.search(pattern, assertion):
                        assertion_patterns[i].add(f"{category}:{name}")
                        pattern_counts[category][name] += 1
                        matches_found = True

            if not matches_found:
                uncategorized_assertions.append(assertion)

        # Calculate metrics
        results = {}
        for category, patterns in pattern_counts.items():
            category_assertions = len([i for i in assertion_patterns.values()
                                    if any(p.startswith(f"{category}:") for p in i)])
            results[category] = {
                'total_matches': category_assertions,
                'coverage_ratio': category_assertions / total_assertions,
                'pattern_breakdown': dict(patterns)
            }

        # Add overall metrics
        results['overall'] = {
            'total_assertions': total_assertions,
            'patterns_per_assertion': sum(len(p) for p in assertion_patterns.values()) / total_assertions,
            'pattern_coverage': len([p for p in sum([list(p.values()) for p in pattern_counts.values()], []) if p > 0]) / \
                              len(sum([list(p.values()) for p in self.patterns.values()], [])),
            'uncategorized': len(uncategorized_assertions),
            'uncategorized_assertions': uncategorized_assertions
        }

        return results

In [22]:
pattern_analyzer = CoveragePatternAnalyzer()

In [23]:
def print_pattern_analyzer_results(pattern_analyzer, dataset, test_suites):
  test_suites_str = "\n\n".join(test_suites)
  pattern_analyzer_results = pattern_analyzer.analyze_test_suite(test_suites_str)
  for item in pattern_analyzer_results:
    print(item)
    print(pattern_analyzer_results[item])

In [24]:
print_pattern_analyzer_results(pattern_analyzer, dataset, deepseek_7b_extracted_test_suites)

boundary_testing
{'total_matches': 38, 'coverage_ratio': 0.2878787878787879, 'pattern_breakdown': {'zero_values': 34, 'negative_values': 7, 'large_values': 2}}
functionality
{'total_matches': 50, 'coverage_ratio': 0.3787878787878788, 'pattern_breakdown': {'complex_input': 50}}
error_handling
{'total_matches': 23, 'coverage_ratio': 0.17424242424242425, 'pattern_breakdown': {'exception_testing': 23}}
edge_cases
{'total_matches': 33, 'coverage_ratio': 0.25, 'pattern_breakdown': {'empty_input': 29, 'single_element': 5, 'null_input': 1}}
overall
{'total_assertions': 132, 'patterns_per_assertion': 1.143939393939394, 'pattern_coverage': 0.8, 'uncategorized': 22, 'uncategorized_assertions': ['assert end - start < 1  # This test should finish in less than 1 second', "assert is_palindrome('x') == True", "assert is_palindrome('xyx') == True", "assert is_palindrome('jerry') == False", "assert make_palindrome('x') == 'x'", "assert make_palindrome('xyz') == 'xyzyx'", "assert make_palindrome('xyx') =

In [25]:
print_pattern_analyzer_results(pattern_analyzer, dataset, deepseek_6_7b_extracted_test_suites)

boundary_testing
{'total_matches': 43, 'coverage_ratio': 0.21393034825870647, 'pattern_breakdown': {'zero_values': 21, 'negative_values': 16, 'large_values': 11}}
functionality
{'total_matches': 64, 'coverage_ratio': 0.31840796019900497, 'pattern_breakdown': {'complex_input': 64}}
edge_cases
{'total_matches': 40, 'coverage_ratio': 0.19900497512437812, 'pattern_breakdown': {'empty_input': 31, 'single_element': 10, 'null_input': 2}}
error_handling
{'total_matches': 43, 'coverage_ratio': 0.21393034825870647, 'pattern_breakdown': {'exception_testing': 43}}
overall
{'total_assertions': 201, 'patterns_per_assertion': 0.9850746268656716, 'pattern_coverage': 0.8, 'uncategorized': 45, 'uncategorized_assertions': ["assert is_palindrome('a') == True", "assert is_palindrome('aa') == True", "assert is_palindrome('abc') == False", "assert is_palindrome('abcba') == True", "assert is_palindrome('abcd') == False", "assert make_palindrome('x') == 'x'", "assert make_palindrome('xyz') == 'xyzyx'", "assert

In [26]:
print_pattern_analyzer_results(pattern_analyzer, dataset, semcoder_extracted_test_suites)

edge_cases
{'total_matches': 206, 'coverage_ratio': 0.47139588100686497, 'pattern_breakdown': {'null_input': 100, 'empty_input': 106, 'single_element': 1}}
boundary_testing
{'total_matches': 8, 'coverage_ratio': 0.018306636155606407, 'pattern_breakdown': {'zero_values': 6, 'negative_values': 4}}
functionality
{'total_matches': 9, 'coverage_ratio': 0.020594965675057208, 'pattern_breakdown': {'complex_input': 9}}
overall
{'total_assertions': 437, 'patterns_per_assertion': 0.517162471395881, 'pattern_coverage': 0.6, 'uncategorized': 218, 'uncategorized_assertions': ['assert hello("Alice") == "Hello, Alice"', 'assert hello("Bob") == "Hello, Bob"', 'assert hello("Alice") == "Hello, Alice"', 'assert hello("Bob") == "Hello, Bob"', 'assert hello("Alice") == "Hello, Alice"', 'assert hello("Bob") == "Hello, Bob"', 'assert hello("Alice") == "Hello, Alice"', 'assert hello("Bob") == "Hello, Bob"', 'assert hello("Alice") == "Hello, Alice"', 'assert hello("Bob") == "Hello, Bob"', 'assert hello("Alice

### GPT-4 Results

#### Test Case Generation

In [32]:
generate_humaneval_tests("gpt-4", num_total_tests=100)

Generated 3 enhanced tests
Total tests so far: 3/100

Test prompt:

Please provide executable test cases for this function:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


Working test examples:
['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True', 'assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False', 'assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False']

Include these types of tests:
1. Performance test:
def test_h

([{'problem_id': 0,
   'entry_point': 'has_close_elements',
   'tests': 'def test_has_close_elements():\n    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\n    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n    assert has_close_elements([], 0.5) == False\n\ndef test_has_close_elements_perf():\n    import time\n    start = time.time()\n    assert has_close_elements([1.0]*10000 + [2.0], 0.5) == True\n    end = time.time()\n    assert end - start

In [33]:
gpt_4_extracted_test_suites = process_file_path("/content/gpt-4_test_case_generation_results.txt")

ORIGINAL TEST SUITE:
def test_has_close_elements():
    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False
    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False
    assert has_close_elements([], 0.5) == False

def test_has_close_elements_perf():
    import time
    start = time.time()
    assert has_close_elements([1.0]*10000 + [2.0], 0.5) == True
    end = time.time()
    assert end - start < 1

def test_has_close_elements_error():
    import pytest
    with

#### GPT Novelty

In [44]:
gpt_4_novelty_results = get_novelty_results(dataset, gpt_4_extracted_test_suites)
print(calculate_aggregate_metrics(gpt_4_novelty_results, "novelty_score"))

{'mean_novelty_score': 0.75, 'median_novelty_score': 0.75, 'min_novelty_score': 0.7, 'max_novelty_score': 0.8, 'std_dev': 0.051176631571915945, 'total_entries_analyzed': 22}


In [45]:
write_results_to_file(gpt_4_novelty_results, 'gpt_4_novelty_results.json')
files.download('gpt_4_novelty_results.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>